In [1]:
# pull in quality data from Wikimedia
from paws.EpochFail import wikiquality as wq

In [2]:
# check to make sure we got what we expected
print(next(wq.read_rows('en')))

{'weighted_sum': 0.10413085831157592, 'title': 'Lost Moon', 'timestamp': '20051201000000', 'rev_id': 28847753, 'prediction': 'Stub', 'page_id': 3204489}


In [3]:
# here's what those 'predicted' values mean: 
# https://meta.wikimedia.org/wiki/Objective_Revision_Evaluation_Service#Article_quality_models


In [9]:
# look for the women scientists
print(next(wq.read_page_ids('women scientists')))

{'page_id': 9622178}


In [80]:
def page_quality(page_id_set):
    page_by_month = {} 
    for i in wq.read_aq('en'):
        while len(page_by_month) < 3:
            if i['page_id'] in page_id_set:
                if i['page_id'] not in page_by_month:
                    page_by_month[i['page_id']] = {}
                time = i['timestamp']
                new_val = dict(i)
                del new_val['timestamp']
                del new_Val['page_id']
                page_by_month[i['page_id']][time] = new_val
            
    return page_by_month
                
def get_page_ids(category, file_name):
    page_id_set = set()
    generator = wq.read_page_ids(category)
    for page in generator:
        page_id_set.add(page['page_id'])
    with open(file_name, 'w') as f:
        for page_id in page_id_set:
            f.write(str(page_id) + '\n')
    return page_id_set

In [81]:
women_scientists = get_page_ids('women scientists', 'data.txt')

In [82]:
page_quality(women_scientists)

KeyboardInterrupt: 